<a href="https://colab.research.google.com/github/Ellakiya15/dijikstra-algo/blob/main/gmap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import networkx as nx
import folium
from heapq import heappop, heappush
from geopy.distance import geodesic
df = pd.read_csv('dataset.csv')

In [2]:
#A*
def haversine(coord1, coord2):
    return geodesic(coord1, coord2).km

def a_star(graph, start, goal):
    open_set = []
    heappush(open_set, (0, start))

    g_costs = {start: 0}
    f_costs = {start: haversine(graph.nodes[start]['pos'], graph.nodes[goal]['pos'])}
    came_from = {}

    while open_set:
        current_f_cost, current = heappop(open_set)

        if current == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            path.reverse()
            return path

        for neighbor in graph.neighbors(current):
            tentative_g_cost = g_costs[current] + haversine(graph.nodes[current]['pos'], graph.nodes[neighbor]['pos'])

            if neighbor not in g_costs or tentative_g_cost < g_costs[neighbor]:
                came_from[neighbor] = current
                g_costs[neighbor] = tentative_g_cost
                f_costs[neighbor] = tentative_g_cost + haversine(graph.nodes[neighbor]['pos'], graph.nodes[goal]['pos'])
                heappush(open_set, (f_costs[neighbor], neighbor))
    return None


In [3]:
G = nx.Graph()
for _, row in df.iterrows():
    G.add_node(row['Postcode'], pos=(row['Latitude'], row['Longitude']))

for i, row1 in df.iterrows():
    for j, row2 in df.iterrows():
        if i != j:
            distance = haversine((row1['Latitude'], row1['Longitude']), (row2['Latitude'], row2['Longitude']))
            if distance < 0.5:  #threshold
                G.add_edge(row1['Postcode'], row2['Postcode'], weight=distance)

# Function to create a Folium map
def create_map(graph, start=None, goal=None, path=None):
    # Set the initial location to the average of all postcodes
    initial_location = [df['Latitude'].mean(), df['Longitude'].mean()]
    map_folium = folium.Map(location=initial_location, zoom_start=14)

    # Add markers for all postcodes
    for node in graph.nodes():
        folium.Marker(
            location=[graph.nodes[node]['pos'][0], graph.nodes[node]['pos'][1]],
            popup=node,
            icon=folium.Icon(color='blue' if node not in (start, goal) else 'green')
        ).add_to(map_folium)

    # If a path is provided, draw the path
    if path:
        path_coordinates = [(graph.nodes[node]['pos'][0], graph.nodes[node]['pos'][1]) for node in path]
        folium.PolyLine(path_coordinates, color='red', weight=5).add_to(map_folium)

    return map_folium

In [4]:
# Define start and end postcodes
#start_postcode = input("Enter the start postcode:")
#end_postcode = input("Enter the end postcode")
start_postcode = 'E18 2LE'
end_postcode = 'E18 1PA'
shortest_path = a_star(G, start_postcode, end_postcode)

if shortest_path:
    print(f"Shortest path from {start_postcode} to {end_postcode}:")
    print(" -> ".join(shortest_path))
    map_folium = create_map(G, start=start_postcode, goal=end_postcode, path=shortest_path)
    map_folium.save("shortest_path_map.html")
    print("Map saved as 'shortest_path_map.html'")
else:
    print(f"No path found between {start_postcode} and {end_postcode}.")

Shortest path from E18 2LE to E18 1PA:
E18 2LE -> E18 2HX -> E18 2HR -> E18 2EB -> E18 1PR -> E18 1PA
Map saved as 'shortest_path_map.html'
